In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import pandas as pd
import numpy as np

import os
import warnings

from scipy.io import wavfile

from hmmlearn import hmm  
import pomegranate

from python_speech_features import mfcc

from sklearn.externals import joblib

import hmm_model_feature_extraction

import random

import matplotlib.pyplot as plt

from tqdm import tqdm

from ast import literal_eval

from google.cloud import storage

gcp = True
use_files_locally = True

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
def score_one_sample(trained_models, test_file_name):
    
    """
    trained_model: ModelHMM object with trained model
    test_file_path: path to wav file
    """
    # empty list to hold all of the scores
    scores = []
    
    # load in file from validation set and convert to mfcc features
    fe = hmm_model_feature_extraction.feature_extraction(mixed_dir="../../../validation/", sampling_freq = 20000, gcp = False)
    loaded = fe.read(test_file_name)
    mfcc_features = fe.return_mfcc(loaded, nfft=1200)
    
    # iterate through each of the trained models
    for i in trained_models:
        
        # compute log likelihood score for using each of the trained models
        sample_score = i[0][0][0].compute_score(mfcc_features)
        scores.append(sample_score)
        
    predicted = scores.index(max(scores))
    print("max score is:", max(scores), "at index:", predicted)
    predicted = trained_models[predicted][0][1]
   
    return predicted

In [ ]:
# load model from pkl

if gcp == True:
    
    with fs.open('ad-bucket-15730/hmm_models/model_{}.pkl'.format(trial_name), 'rb') as file:
        asdf = pickle.load(file)
        
else:

    trained_models = joblib.load(hmm_model_dir+'/model_{}.pkl'.format(trial_name))

In [ ]:
trained_models

In [ ]:
# load in validation samples:

if use_files_locally == True:
    
    f = open('ad-bucket-15730/hmm_models/validation_samples_{}.txt'.format(trial_name),'r')
    validation_samples = f.readlines()
    f.close()
    validation_samples = [literal_eval(validation_samples[i]) for i in range(len(validation_samples))]    

else:
    
    f = fs.open('ad-bucket-15730/hmm_models/validation_samples_{}.txt'.format(trial_name),'r')
    validation_samples = f.readlines()
    f.close()
    validation_samples = [literal_eval(validation_samples[i]) for i in range(len(validation_samples))]

In [ ]:
points = 0
validation_list = []

for i in tqdm(range(len(validation_samples))):
    
    print("validation sample size: ", len(validation_samples))
    actual = validation_samples[i][1]
    print("actual:", actual)
    predicted = score_one_sample(trained_models, validation_samples[i][0])
    print("predicted:", predicted)
    validation_list.append((actual,predicted))
    if actual == predicted:
        
        points += 1
        print("scored a point!")
        print(points, "points")

In [ ]:
# print overall accuracy

overall_accuracy = points/len(validation_samples)
overall_accuracy

In [ ]:
validation_performance_df = pd.DataFrame(validation_list, columns = ['actual', 'predicted']) 

In [ ]:
# save validation performance to hmm_models

if gcp == True:
    
    validation_performance_df.to_csv(hmm_model_dir + '/validation_performance_{}.csv'.format(trial_name))
    label_states.to_csv(hmm_model_dir + '/label_states_{}.csv'.format(trial_name))
    
else:
    
    validation_performance_df.to_csv(hmm_model_dir + '/validation_performance_{}.csv'.format(trial_name))
    label_states.to_csv(hmm_model_dir + '/label_states_{}.csv'.format(trial_name))